In [31]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re

In [32]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
#loading the necessary libraries packages


In [33]:
class DataLoader:
  def __init__(self,dir_name,file_name):
    self.dir_name=dir_name
    self.file_name = file_name
    
 
  def read_csv(self):
    os.chdir(self.dir_name)
    tweets_df=pd.read_csv(self.file_name)
    return tweets_df
  # this is a class to load teh data

In [34]:
DataLoader_obj= DataLoader('../Wednsday_Task','cleaned_tweet_data.csv')

In [35]:
tweets_df=DataLoader_obj.read_csv()
tweets_df.dropna()


,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,clean_text
0,2021-06-18 17:55:49+00:00,http://twitter.com/download/iphone,"🚨Africa is ""in the midst of a full-blown third...",0.166667,0.188889,en,548,612.0,ketuesriche,551,351,0.0,,TelGlobalHealth WHOAFRO,mass,"africa is ""in the midst of a full-blown third ..."
1,2021-06-18 17:55:59+00:00,https://mobile.twitter.com,"Dr Moeti is head of WHO in Africa, and one of ...",0.133333,0.455556,en,195,92.0,Grid1949,66,92,0.0,,globalhlthtwit,"edinburgh, scotland",dr moeti is head of who in africa and one of t...
2,2021-06-18 17:56:07+00:00,http://twitter.com/download/iphone,Thank you @research2note for creating this ama...,0.316667,0.483333,en,2,1.0,LeeTomlinson8,1195,1176,0.0,red4research,NHSRDForum Research2note NHSRDForum,,thank you for creating this amazing campaign ...
3,2021-06-18 17:56:10+00:00,https://mobile.twitter.com,"Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,0.197222,en,1580,899.0,RIPNY08,2666,2704,0.0,,HighWireTalk,,former pfizer vp and virologist dr michael yea...
4,2021-06-18 17:56:20+00:00,http://twitter.com/download/android,I think it’s important that we don’t sell COVA...,0.280000,0.620000,en,72,20.0,pash22,28250,30819,0.0,,PeterHotez,united kingdom,i think it s important that we don t sell cova...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6527,2021-06-19 07:41:15+00:00,http://twitter.com/download/android,RT @Givenkazeni: Zweli please just release the...,0.000000,0.400000,en,188,54.0,Mthatos_Vivi,447,1089,0.0,,Givenkazeni,,rt : zweli please just release the graphic des...
6528,2021-06-19 07:41:26+00:00,http://twitter.com/download/android,"Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,0.197222,en,1595,908.0,wayno_af007,2224,2739,0.0,,HighWireTalk,"the boro, ma",former pfizer vp and virologist dr michael yea...
6529,2021-06-19 07:41:31+00:00,http://twitter.com/download/iphone,"@Jenfeds73 @DcrInYYC Respectfully, veterinaria...",0.281250,0.506250,en,0,0.0,dublonothing,3000,4709,0.0,,Jenfeds73 DcrInYYC,"los angeles, ca",respectfully veterinarians have to have a broa...
6530,2021-06-19 07:41:45+00:00,http://twitter.com/download/iphone,"""Africa needs millions more doses here &amp; n...",0.125000,0.325000,en,143,122.0,DrAmirKhanGP,135163,1284,0.0,covid19,WHOAFRO,yorkshire and the humber,"""africa needs millions more doses here &amp; n..."


In [36]:
len(tweets_df)


6532

In [37]:
class PrepareData:
  def __init__(self,df):
    self.df=df
    
  def preprocess_data(self):
    tweets_df = self.df.loc[self.df['lang'] =="en"]

    
    #text Preprocessing
    tweets_df['clean_text']=tweets_df['clean_text'].astype(str)
    tweets_df['clean_text'] = tweets_df['clean_text'].apply(lambda x: x.lower())
    tweets_df['clean_text']= tweets_df['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))
    
    #Converting tweets to list of words For feature engineering
    sentence_list = [tweet for tweet in tweets_df['clean_text']]
    word_list = [sent.split() for sent in sentence_list]

    #Create dictionary which contains Id and word 
    word_to_id = corpora.Dictionary(word_list)
    corpus_1= [word_to_id.doc2bow(tweet) for tweet in word_list]



    
    return word_list, word_to_id, corpus_1

In [38]:
PrepareData_obj=PrepareData(tweets_df)
word_list ,id2word,corpus=PrepareData_obj.preprocess_data()


In [39]:
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 3), (22, 1), (23, 2), (24, 1), (25, 4), (26, 1), (27, 1), (28, 1), (29, 1)], [(2, 1), (4, 3), (6, 4), (15, 1), (16, 2), (17, 2), (21, 2), (25, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 3), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1)], [(2, 1), (17, 1), (21, 2), (25, 2), (35, 2), (60, 1), (61, 2), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1)], [(6, 1), (17, 1), (21, 3), (25, 3), (32, 1), (44, 1), (56, 1), (60, 1), (87, 1), (88, 1), (89, 1), (90, 1),

In [26]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [27]:
pprint(lda_model.show_topics(formatted=False))

[(0,
  [('capacity', 0.04463504),
   ('vaccines', 0.042004604),
   ('age', 0.04121999),
   ('limit', 0.03733404),
   ('pin', 0.03733404),
   ('min', 0.03733404),
   ('dose1', 0.03733404),
   ('covaxin', 0.03171102),
   ('to', 0.027064182),
   ('0', 0.026556812)]),
 (1,
  [('the', 0.052431516),
   ('in', 0.041037105),
   ('india', 0.039802242),
   ('of', 0.0374409),
   ('africa', 0.025605375),
   ('covid19', 0.023522032),
   ('and', 0.01971889),
   ('a', 0.01908519),
   ('amp', 0.014563558),
   ('to', 0.01442402)]),
 (2,
  [('to', 0.051573478),
   ('and', 0.046089526),
   ('the', 0.034761816),
   ('we', 0.028445799),
   ('of', 0.02711056),
   ('need', 0.025345674),
   ('africa', 0.022822494),
   ('vaccines', 0.019880135),
   ('with', 0.017707573),
   ('is', 0.01711468)]),
 (3,
  [('the', 0.043029755),
   ('to', 0.034499854),
   ('covid19', 0.02972901),
   ('of', 0.02518709),
   ('vaccines', 0.023332171),
   ('is', 0.016569745),
   ('in', 0.016371908),
   ('are', 0.016264228),
   ('for',

In [28]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -6.215648355375872

 Ldamodel Coherence Score/Accuracy on Tweets:  0.5477314024559427


In [29]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.118956  0.057252       1        1  39.016667
1      0.221091  0.003824       2        1  22.011762
4     -0.192888 -0.294766       3        1  16.213341
2      0.137431  0.004722       4        1  13.942266
0     -0.284589  0.228968       5        1   8.815964, topic_info=         Term         Freq        Total Category  logprob  loglift
85        you  1822.000000  1822.000000  Default  30.0000  30.0000
274     india  1589.000000  1589.000000  Default  29.0000  29.0000
256        rt  1153.000000  1153.000000  Default  28.0000  28.0000
56   vaccines  4543.000000  4543.000000  Default  27.0000  27.0000
541    israel   983.000000   983.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
141   vaccine   110.587604   990.480784   Topic5  -4.9734   0.2362
146       are   105.852649  1855.755513   Topic5  -5.0171  -0.4354
2           a   105.063610  2011.331005   Topic5  -5.0246  -0.5234
159     there    96.486499   409.259145   Topic5  -5.1098   0.9837
25        the   107.836848  6415.921619   Topic5  -4.9985  -1.6573

[280 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
752       5  0.997123            0
341       2  0.285408            1
341       3  0.713519            1
745       5  0.997765           18
143       5  0.992856  18002320233
...     ...       ...          ...
85        5  0.065299          you
163       3  0.916160         your
163       5  0.082780         your
164       5  0.989752          zip
2854      2  0.995150        zweli

[308 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 5, 3, 1])